### Binary classification

**Rationale**: Predict the sentiment of customer reviews

In [1]:
// ML.NET Nuget packages 
#r "nuget:Microsoft.ML,1.4.0"  
    
//Install XPlot package
#r "nuget:XPlot.Plotly,2.0.0"

using Microsoft.ML;
using Microsoft.ML.Data;
using XPlot.Plotly;

Installing package Microsoft.ML, version 1.4.0.................done!

Successfully added reference to package Microsoft.ML, version 1.4.0

Installing package XPlot.Plotly, version 2.0.0.....done!

Successfully added reference to package XPlot.Plotly, version 2.0.0

Installing package System.Data.SqlClient, version 4.2.0........done!

Successfully added reference to package System.Data.SqlClient, version 4.2.0

In [2]:
using Microsoft.ML.Data;

public class ProductReview 
{   
    [LoadColumn(0)]
    public bool Sentiment;
       
    [LoadColumn(1)]
    public string Review;     
}

In [4]:
var dataPath = "./Datasets/Binary-Classification Sentiment/product_reviews.csv";

var mlContext = new MLContext(seed: 1);

IDataView dataView = mlContext.Data.LoadFromTextFile<ProductReview>(dataPath, hasHeader: true, separatorChar: ',');

#### Data exploration

In [5]:
var rows = mlContext.Data.CreateEnumerable<ProductReview>(dataView, reuseRowObject: false)
                    .Take(10)
                    .ToList();

display(rows);

index,Sentiment,Review
0,True,""" I have read a couple of other disappointing reviews and was so-o-o-o skeptical! I also bought over the years a few of the A/C vent mounts for cell phone and gave up on liking any of them. They always had been flimsy and shaky snapping off in a second... NOT THIS ONE! I have used it for a week and loved every second of it!1. It does stick to the """"pimpled"""" dashboard and doesn't care: I cannot pull it with both hands!2. The phone does stick to the sticky pad. Actually it is a bit of a problem because every time I stop the car and want to remove the phone I have to struggle with the sticky pad: it holds the phone very strongly.3. It does recovery after washing: after it got a bit dusty I washed it gently with a soap rinsed well and air-dried. That was it: works like new.Maybe I got lucky with this unit but it works and I love it. One suggestion: do not throw away a thick plastic cover on the sticky pad for the phone: it will work as a handy protector of the sticky pad when the phone is removed for extended periods of time."""
1,True,Be aware what your device supports! I initially purchased this for a Google/LG Nexus 5 cell phone which does not support MHL. However we also have a Google/Samsung Galaxy Nexus (or Nexus 3) which this cable works brilliantly with.Be aware that MHL and SlimPort are the two competing cable designs and you must know which your phone or tablet can use.
2,True,I have both the Frost Black and Frost White versions of this case. Both are excellent. The Frost Black version does not allow you to see any details from the back of the phone. The Frost White has some transparency on the back so you can still see the Nexus logo but not the reflective glass back. The case allows all ports cameras flash and speaker to be accessible. What I love most about the case is that the material used to cover the volume rocker and power button provide very good feedback and in my opinion make the buttons much easier to push without your finger slipping in the midst of trying to do something else like maneuver the phone. I would not buy a different case with my N4.
3,True,OEM HTC USB Travel Charger Adapte. Ordered this as a spare charger for my HTC Thunderbolt - it looks just like the one that came with my phone but the AC adapter does not work. &#8221;
4,True,The Samsung Note 2 Spare battery charging station is a great addition to my phone with multiple uses.The first day and night I used the Samsung charging station I seen the benefits. Stand served as a great holder for Note 2 for my alarm app. I watched videos on the stand without having to hold my Note 2 in my hands for long periods of time. While watching videos I had installed the spare battery to charge. This was a blessing because the next morning when getting ready for work I found I forgot to plug in phone and had only 25% remaining. I use my phone all day for reports audits pictures emails calls and media. Then I remembered the spare battery inside charging station. This is the only experience I needed to see and know this is a life saver product. Great size portable for when I travel can plug charging station cable straight into phone to charge current battery allowing me to have a backup battery in case of emergencies. I rate this product a 5 star based on personally experiencing the importance of having a backup.
5,False,Lousy. Slips off of the neck. Almost lost them twice in twelve hours. Unless you'd like to use a foolish looking and uncomfortable clip. No thanks. Earbuds are bulky and unwieldy. Controls are limited to 3 buttons which have a number of combinations that are supposed to control everything. What a pain! I never did get them to do half of what they were supposed to. Sent them back.
6,True,These are so much better than I expected honestly. First off - looking at the photos I thought the neck band would be stiff but it's not! It's super soft and wraps nicely around my head. Honestly I fell asleep with these on lay

#### Plotting

In [6]:
var countPositiveSentiment = dataView.GetColumn<bool>("Sentiment").Count(x => x == true);
var countNegativeSentiment = dataView.GetColumn<bool>("Sentiment").Count(x => x == false);

var values = new List<int> { countPositiveSentiment, countNegativeSentiment};
var labels = new List<string> { "Positive", "Negative" };

var pieChart = Chart.Plot(new Graph.Pie 
{
    labels = labels,
    values = values
});

pieChart

#### Split our data

In [7]:
var trainTestSplit = mlContext.Data.TrainTestSplit(dataView);

#### Data Transformation

In [8]:
var dataProcessPipeline = mlContext.Transforms.Text.FeaturizeText("Review")
        .Append(mlContext.Transforms.CopyColumns("Features", "Review"))
        .Append(mlContext.Transforms.NormalizeMinMax("Features", "Features"));

In [9]:
var trainer = mlContext.BinaryClassification.Trainers.LbfgsLogisticRegression(labelColumnName: "Sentiment", featureColumnName: "Features");

var trainingPipeline = dataProcessPipeline.Append(trainer);

##### Train the model

In [10]:
ITransformer model = trainingPipeline.Fit(trainTestSplit.TrainSet);

#### Evaluate the model

In [11]:
var predicitions = model.Transform(trainTestSplit.TestSet);
var metrics = mlContext.BinaryClassification.Evaluate(predicitions, labelColumnName: "Sentiment");

display("Recall:" + " " + metrics.PositiveRecall);
display("Accuracy:" + " " + metrics.Accuracy);
display("F1Score:" + " " + metrics.F1Score); 

Recall: 0.9863189789734643

Accuracy: 0.9590931493346476

F1Score: 0.9773451005368029

#### Save the model

In [12]:
mlContext.Model.Save(model, trainTestSplit.TrainSet.Schema, "sentimentClassifier.zip");